In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords  #stopwords
from nltk.stem import WordNetLemmatizer  
from nltk.tokenize import word_tokenize
#from sklearn.feature_extraction.text import TfidfVectorizer



filenames = ['pizza_reviews_file1.csv', 'pizza_reviews_file2.csv', 'pizza_reviews_file3.csv', 'pizza_reviews_file4.csv']
rev = pd.concat([pd.read_csv(file) for file in filenames], ignore_index=True)

print(rev.head())

   Unnamed: 0.2  Unnamed: 0.1  Unnamed: 0        restaurant    city  \
0             0             0         0.0  Home Slice Pizza  Austin   
1             1             1         1.0  Home Slice Pizza  Austin   
2             2             2         2.0  Home Slice Pizza  Austin   
3             3             3         3.0  Home Slice Pizza  Austin   
4             4             4         4.0  Home Slice Pizza  Austin   

                                              review  
0  Somehow I lived in Austin for 22 years before ...  
1  Always the go to for a slice in Austin. Super ...  
2  We went on a Sunday night around 9:30pm during...  
3  Came in here this past Saturday afternoon arou...  
4  Without a side by side comparison available I ...  


In [2]:
df = rev[['city','restaurant','review']]

In [3]:
import nltk
from nltk.corpus import stopwords  #stopwords
from nltk.stem import WordNetLemmatizer  
from sklearn.feature_extraction.text import TfidfVectorizer
stop_words=set(nltk.corpus.stopwords.words('english'))

In [4]:
import nltk
nltk.download('omw-1.4')


[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/uditdhand/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [5]:
def clean_text(headline):
      le=WordNetLemmatizer()
      word_tokens=word_tokenize(headline)
      tokens=[le.lemmatize(w) for w in word_tokens if w not in stop_words and len(w)>3]
      cleaned_text=" ".join(tokens)
      return cleaned_text

df['cleaned_text']=df['review'].apply(clean_text)

/var/folders/hg/kqg69xz568l28vnvcx6x09hh0000gn/T/ipykernel_71979/3790745193.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cleaned_text']=df['review'].apply(clean_text)


In [8]:
vect = TfidfVectorizer(stop_words=list(stop_words), max_features=1000)
vect_text=vect.fit_transform(df['cleaned_text'])

In [19]:
from sklearn.decomposition import LatentDirichletAllocation
lda_model=LatentDirichletAllocation(n_components=10,
learning_method='online',random_state=42,max_iter=100) 
lda_top=lda_model.fit_transform(vect_text)

In [20]:
print("Document 0: ")
for i,topic in enumerate(lda_top[0]):
  print("Topic ",i,": ",topic*100,"%")

Document 0: 
Topic  0 :  1.506341852159411 %
Topic  1 :  6.758228684898897 %
Topic  2 :  1.4766144204050073 %
Topic  3 :  1.4766376428614434 %
Topic  4 :  1.4763482968761255 %
Topic  5 :  1.476606272692909 %
Topic  6 :  1.477650081562217 %
Topic  7 :  81.3984497338227 %
Topic  8 :  1.4767773569611664 %
Topic  9 :  1.476345657760139 %


In [21]:
vocab = vect.get_feature_names_out()
for i, comp in enumerate(lda_model.components_):
     vocab_comp = zip(vocab, comp)
     sorted_words = sorted(vocab_comp, key= lambda x:x[1], reverse=True)[:10]
     print("\nTopic "+str(i)+": ")
     for t in sorted_words:
            print(t[0],end=" ")



Topic 0: 
seattle free gluten gino burger fry pinthouse hole craft worker 
Topic 1: 
sandwich miami nonna vegan soup miss regret hand world vegetarian 
Topic 2: 
crust pequod sauce cheese slice pepperoni thin pizza crispy plain 
Topic 3: 
pizza slice good wait pepperoni worth crust best great place 
Topic 4: 
vodka detroit cake potato pudding bolognese girlfriend bacon meatball style 
Topic 5: 
great pizza dish deep chicago place beer good food service 
Topic 6: 
pizza good place dish time food great deep ordered wait 
Topic 7: 
pizza slice place best good pepperoni like crust line square 
Topic 8: 
great pizza service food best amazing recommend place staff highly 
Topic 9: 
truffle prosciutto calamari beautiful pike fried risotto wood glass fired 

For this set of topics, the interpretations can be as follows:

### Topic Interpretations:
1. **Topic 0:**
   - **Theme:** Dietary Preferences and Seattle's Craft Food.
   - **Description:** Focused on Seattle and mentions "gluten-free" options, suggesting dietary preferences. Also mentions "craft," possibly related to craft beers or artisanal food, and specific eateries like “Gino” and “Pinthouse.”

2. **Topic 1:**
   - **Theme:** Vegetarian/Vegan Options and Miami's Cuisine.
   - **Description:** References to vegetarian and vegan options like "vegan soup," with mention of Miami and “Nonna,” which might be a specific location or a term for grandmother, possibly reflecting traditional recipes.

3. **Topic 2:**
   - **Theme:** Pizza Characteristics.
   - **Description:** This topic is likely about the qualities and types of pizzas, focusing on crust, sauce, cheese, and toppings like pepperoni. Words like “crispy” and “plain” describe pizza features.

4. **Topic 3:**
   - **Theme:** Quality and Worth of Pizza.
   - **Description:** This topic primarily discusses the goodness and worthiness of waiting for pizza. It covers elements like crust, slices, and pepperoni, denoting different pizza types and their quality.

5. **Topic 4:**
   - **Theme:** Variety in Menu and Detroit's Cuisine.
   - **Description:** Discussing a diverse range of foods like “cake,” “pudding,” “bacon,” “meatball,” and mentions “Detroit,” possibly hinting at specialty dishes or styles from Detroit.

6. **Topic 5:**
   - **Theme:** Chicago’s Deep Dish and Dining Experience.
   - **Description:** Focuses on Chicago’s deep-dish pizza and the overall dining experience, discussing the quality of food, beer, and service.

7. **Topic 6:**
   - **Theme:** Overall Dining and Ordering Experience.
   - **Description:** Discussions revolve around the quality of food, waiting times, and ordered items, possibly reflecting the overall satisfaction or dissatisfaction with the service and food.

8. **Topic 7:**
   - **Theme:** Pizza Quality and Preferences.
   - **Description:** Focuses on the goodness of pizza slices and crust, with mentions of “best,” indicating discussions on personal preferences and the comparison of quality.

9. **Topic 8:**
   - **Theme:** Highly Recommended Service and Food.
   - **Description:** Words like “recommend,” “amazing,” “staff,” “highly,” suggest this topic revolves around the exceptional service and food quality at certain places.

10. **Topic 9:**
    - **Theme:** Gourmet Ingredients and Cooking Style.
    - **Description:** Mentions gourmet ingredients like “truffle” and “prosciutto” and cooking styles like “wood-fired,” indicating discussions about the specialty items and quality of cooking.

### Utilization:
- These interpretations can help in understanding regional variations in pizza preferences and dining experiences.
- The mention of specific cities (e.g., Miami, Detroit, Seattle, and Chicago) can help in understanding the specialties or preferences of those cities or comparing pizza places in different regions.
- Additionally, the occurrence of words related to specific dietary needs like "vegan," "vegetarian," and "gluten-free" highlights the attention given to diverse food requirements in the reviews.

### Further Analysis:
- You can delve deeper into each city's specific mentions to see if there is any regional preference or a regional specialty that is highly liked or disliked.
- Analyzing the sentiment of the reviews corresponding to each topic will provide more context on whether the topics are discussed in a positive, negative, or neutral manner.